In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = "YOUR_ACCESS_KEY_ID_HERE"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YOUR_SECRET_ACCESS_KEY_HERE"
os.environ["AWS_DEFAULT_REGION"] = "me-central-1"

print(os.environ["AWS_ACCESS_KEY_ID"][:6])


In [ ]:
!pip install boto3


In [ ]:
import boto3

iam = boto3.client("iam")
sts = boto3.client("sts")

In [ ]:
# test the connection
identity = sts.get_caller_identity()
print(identity)

In [ ]:
users = iam.list_users()["Users"]
roles = iam.list_roles()["Roles"]

print("Users:")
for u in users:
    print("-", u["UserName"], u["Arn"])

print("\nRoles:")
for r in roles:
    print("-", r["RoleName"], r["Arn"])

In [ ]:
def get_policies_for_user(user_name):
    policies = []

    # Managed policies attached
    attached = iam.list_attached_user_policies(UserName=user_name)["AttachedPolicies"]
    for p in attached:
        # For each attached policy, get its document (JSON)
        policy_arn = p["PolicyArn"]
        meta = iam.get_policy(PolicyArn=policy_arn)["Policy"]
        version = meta["DefaultVersionId"]
        doc = iam.get_policy_version(
            PolicyArn=policy_arn,
            VersionId=version
        )["PolicyVersion"]["Document"]
        policies.append(doc)

    # Inline policies directly attached to this user
    inline_names = iam.list_user_policies(UserName=user_name)["PolicyNames"]
    for name in inline_names:
        doc = iam.get_user_policy(
            UserName=user_name,
            PolicyName=name
        )["PolicyDocument"]
        policies.append(doc)

    return policies

In [ ]:
def find_assume_roles_in_policy(policy_doc):
    results = []  # each item: (action, resource)

    statements = policy_doc.get("Statement", [])
    # make sure it's a list
    if isinstance(statements, dict):
        statements = [statements]

    for stmt in statements:
        if stmt.get("Effect") != "Allow":
            continue

        actions = stmt.get("Action", [])
        if isinstance(actions, str):
            actions = [actions]

        if "sts:AssumeRole" not in actions and "sts:*" not in actions and "*" not in actions:
            continue

        resources = stmt.get("Resource", [])
        if isinstance(resources, str):
            resources = [resources]

        for r in resources:
            results.append(("sts:AssumeRole", r))

    return results

In [ ]:
import networkx as nx
G = nx.DiGraph()

# add nodes
for u in users:
    G.add_node(u["Arn"], kind="user", label=u["UserName"])

for r in roles:
    G.add_node(r["Arn"], kind="role", label=r["RoleName"])

# for each user, add edges based on AssumeRole statements
for u in users:
    name = u["UserName"]
    arn = u["Arn"]
    policies = get_policies_for_user(name)
    for pdoc in policies:
        assume_edges = find_assume_roles_in_policy(pdoc)
        for action, resource_arn in assume_edges:
            # if Resource is a role in our account:
            for r in roles:
                if r["Arn"] == resource_arn or resource_arn == "*":
                    G.add_edge(arn, r["Arn"], action=action)

In [ ]:
def get_policies_for_role(role_name):
    policies = []

    # Managed policies attached to the role
    attached = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
    for p in attached:
        policy_arn = p["PolicyArn"]
        meta = iam.get_policy(PolicyArn=policy_arn)["Policy"]
        version = meta["DefaultVersionId"]
        doc = iam.get_policy_version(
            PolicyArn=policy_arn,
            VersionId=version
        )["PolicyVersion"]["Document"]
        policies.append(doc)

    # Inline policies directly on the role
    inline_names = iam.list_role_policies(RoleName=role_name)["PolicyNames"]
    for name in inline_names:
        doc = iam.get_role_policy(
            RoleName=role_name,
            PolicyName=name
        )["PolicyDocument"]
        policies.append(doc)

    return policies


In [ ]:
# Add edges from ROLES to ROLES based on their policies
for r in roles:
    r_name = r["RoleName"]
    r_arn = r["Arn"]

    policies = get_policies_for_role(r_name)
    for pdoc in policies:
        assume_edges = find_assume_roles_in_policy(pdoc)
        for action, resource_arn in assume_edges:
            # if Resource is a role in our account:
            for r2 in roles:
                if r2["Arn"] == resource_arn or resource_arn == "*":
                    G.add_edge(r_arn, r2["Arn"], action=action)

print("Graph now has:")
print("Nodes:", G.number_of_nodes())
print("Edges:", G.number_of_edges())


In [ ]:
# Build a map from ARN-> short name (username or rolename) for pretty printing
name_map = {}

for u in users:
    name_map[u["Arn"]] = u["UserName"]

for r in roles:
    name_map[r["Arn"]] = r["RoleName"]

# Find the ARN of student-user and AdminRole
start_arn = None
target_arn = None

for u in users:
    if u["UserName"] == "student-user":
        start_arn = u["Arn"]
        break

for r in roles:
    if r["RoleName"] == "AdminRole":
        target_arn = r["Arn"]
        break

print("start ARN:", start_arn)
print("target ARN:", target_arn)

import networkx as nx

if start_arn is None or target_arn is None:
    print("Check names: student-user or AdminRole not found.")
else:
    try:
        path = nx.shortest_path(G, start_arn, target_arn)
        print("RAW ARN PATH:")
        for p in path:
            print(" ", p)

        print("\nLADDER (friendly names):")
        print(" -> ".join(name_map[p] for p in path))
    except nx.NetworkXNoPath:
        print("No path from student-user to AdminRole")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))

# layout positions for nodes
pos = nx.spring_layout(G, k=0.8, seed=3)

# draw all nodes
nx.draw_networkx_nodes(G, pos, node_size=800)


nx.draw_networkx_edges(
    G, pos,
    alpha=0.3,
    arrows=True,
    arrowstyle='-|>',
    arrowsize=22
)


nx.draw_networkx_labels(G, pos, labels=name_map, font_size=8)


try:
    path_edges = list(zip(path, path[1:]))
    nx.draw_networkx_edges(
        G, pos,
        edgelist=path_edges,
        edge_color="red",
        width=2,
        arrows=True,
        arrowstyle='-|>',
        arrowsize=26
    )
except NameError:
    pass

plt.title("IAM Privilege Escalation Graph (red = student-user → AdminRole path)")
plt.axis("off")
plt.show()
